In [15]:
import pandas as pd
from collections import Counter
import io
# Загружаем данные в DataFrame
users_df = pd.read_csv('user.csv', sep=';')
appeals_df = pd.read_csv('854.csv', sep=';')

print("Данные пользователей:")
print(users_df)
print("\nДанные обращений:")
print(appeals_df)

# Создаем словарь для сопоставления логина с ФИО
login_to_name = dict(zip(users_df['логин'], users_df['ФИО']))
# Функция для подсчета участия пользователей в обращениях
def count_user_participation(appeals_df):
    service_user_stats = {}
    
    for _, row in appeals_df.iterrows():
        service = row['код услуги']
        users_str = row['Все пользователи']
        
        # Разделяем пользователей и подсчитываем количество участий каждого
        users_list = users_str.split(',')
        user_counts = Counter(users_list)
        
        # Инициализируем счетчик для услуги, если ее еще нет
        if service not in service_user_stats:
            service_user_stats[service] = Counter()
        
        # Добавляем подсчеты для текущего обращения
        service_user_stats[service].update(user_counts)
    
    return service_user_stats

# Подсчитываем статистику
service_stats = count_user_participation(appeals_df)

# Создаем рейтинг для каждой услуги
results = []

for service, user_counter in service_stats.items():
    # Сортируем пользователей по количеству участий (по убыванию)
    sorted_users = user_counter.most_common()
    
    for login, count in sorted_users:
        # Получаем ФИО, если пользователь есть в справочнике
        full_name = login_to_name.get(login, f"Неизвестный ({login})")
        results.append({
            'Услуга': service,
            'Логин': login,
            'ФИО': full_name,
            'Количество_участий': count
        })

# Создаем DataFrame с результатами
results_df = pd.DataFrame(results)

# Выводим рейтинг по каждой услуге
print("\nРейтинг пользователей по каждой услуге:")
for service in results_df['Услуга'].unique():
    print(f"\n--- Услуга: {service} ---")
    service_df = results_df[results_df['Услуга'] == service].sort_values('Количество_участий', ascending=False)
    print(service_df.to_string(index=False))





Данные пользователей:
     логин         ФИО
0   petrov   Петров ПП
1  sidorov  Сидоров АП
2    vanov    Ванов КЕ
3    belov    Белов АП
4    gagoc    Гагоц ВУ

Данные обращений:
    номер обращения код услуги       Все пользователи
0       2025-1256-5         РП   petrov,petrov,petrov
1       2025-4896-5         РП   petrov,ivonov,petrov
2      2025-698-984         КУ  dorov,sidorov,sidorov
3        2025-895-6         ЕП        nov,vanov,vanov
4       2025-5965-5         РП      belov,belov,belov
5      2025-9-6-125         КУ      belov,belov,belov
6     9865-456-9856         РП      gegoc,gegoc,gegoc
7    102025-5-555-1         ЕП      gegoc,gegoc,gegoc
8         2025-89-9         ЕП      gegoc,gegoc,belov
9      2025-1256-51         РП   petrov,petrov,petrov
10     2025-4896-51         РП   petrov,ivonov,petrov
11    2025-698-9841         КУ  dorov,sidorov,sidorov
12      2025-895-61         ЕП        nov,vanov,vanov
13     2025-5965-51         РП      belov,belov,belov
14    2025-

In [16]:
# Альтернативное представление: сводная таблица
print("\nСводная таблица рейтинга:")
pivot_df = results_df.pivot_table(
    index=['ФИО', 'Логин'], 
    columns='Услуга', 
    values='Количество_участий', 
    fill_value=0,
    aggfunc='sum'
)
print(pivot_df)

# Общий рейтинг всех пользователей
print("\nОбщий рейтинг пользователей (по всем услугам):")
total_rating = results_df.groupby(['Логин', 'ФИО'])['Количество_участий'].sum().sort_values(ascending=False)
print(total_rating)


Сводная таблица рейтинга:
Услуга                        ЕП  КУ  РП
ФИО                  Логин              
Белов АП             belov     1   6   6
Ванов КЕ             vanov     4   0   0
Гагоц ВУ             gagoc     4   0   2
Неизвестный (dorov)  dorov     0   2   0
Неизвестный (gegoc)  gegoc     6   0   4
Неизвестный (gаgoc)  gаgoc     1   0   0
Неизвестный (ivonov) ivonov    0   0   2
Неизвестный (nov)    nov       2   0   0
Петров ПП            petrov    0   0  10
Сидоров АП           sidorov   0   4   0

Общий рейтинг пользователей (по всем услугам):
Логин    ФИО                 
belov    Белов АП                13
gegoc    Неизвестный (gegoc)     10
petrov   Петров ПП               10
gagoc    Гагоц ВУ                 6
sidorov  Сидоров АП               4
vanov    Ванов КЕ                 4
dorov    Неизвестный (dorov)      2
ivonov   Неизвестный (ivonov)     2
nov      Неизвестный (nov)        2
gаgoc    Неизвестный (gаgoc)      1
Name: Количество_участий, dtype: int64


In [ ]:
# Более компактное решение с группировкой
# Разбираем столбец с пользователями
exploded_df = appeals_df.copy()
exploded_df['пользователь'] = exploded_df['Все пользователи'].str.split(',')
exploded_df = exploded_df.explode('пользователь')
print(exploded_df)
# Группируем по услуге и пользователю
rating_by_service = exploded_df.groupby(['код услуги', 'пользователь']).size().reset_index(name='количество_участий')

# Добавляем ФИО
rating_by_service['ФИО'] = rating_by_service['пользователь'].map(login_to_name)
rating_by_service['ФИО'] = rating_by_service['ФИО'].fillna(rating_by_service['пользователь'])

# Сортируем по услуге и количеству участий
rating_by_service = rating_by_service.sort_values(['код услуги', 'количество_участий'], ascending=[True, False])

print("Рейтинг по услугам (упрощенная версия):")
for service in rating_by_service['код услуги'].unique():
    print(f"\n--- Услуга: {service} ---")
    service_rating = rating_by_service[rating_by_service['код услуги'] == service]
    print(service_rating[['ФИО', 'пользователь', 'количество_участий']].to_string(index=False))

In [17]:
# Более компактное решение с группировкой
# Разбираем столбец с пользователями
exploded_df = appeals_df.copy()
exploded_df['пользователь'] = exploded_df['Все пользователи'].str.split(',')
exploded_df = exploded_df.explode('пользователь')
print(exploded_df)
# Группируем по услуге и пользователю
rating_by_service = exploded_df.groupby(['код услуги', 'пользователь']).size().reset_index(name='количество_участий')

    номер обращения код услуги       Все пользователи пользователь
0       2025-1256-5         РП   petrov,petrov,petrov       petrov
0       2025-1256-5         РП   petrov,petrov,petrov       petrov
0       2025-1256-5         РП   petrov,petrov,petrov       petrov
1       2025-4896-5         РП   petrov,ivonov,petrov       petrov
1       2025-4896-5         РП   petrov,ivonov,petrov       ivonov
1       2025-4896-5         РП   petrov,ivonov,petrov       petrov
2      2025-698-984         КУ  dorov,sidorov,sidorov        dorov
2      2025-698-984         КУ  dorov,sidorov,sidorov      sidorov
2      2025-698-984         КУ  dorov,sidorov,sidorov      sidorov
3        2025-895-6         ЕП        nov,vanov,vanov          nov
3        2025-895-6         ЕП        nov,vanov,vanov        vanov
3        2025-895-6         ЕП        nov,vanov,vanov        vanov
4       2025-5965-5         РП      belov,belov,belov        belov
4       2025-5965-5         РП      belov,belov,belov        b

In [18]:
# Добавляем ФИО
rating_by_service['ФИО'] = rating_by_service['пользователь'].map(login_to_name)
rating_by_service['ФИО'] = rating_by_service['ФИО'].fillna(rating_by_service['пользователь'])
rating_by_service

,код услуги,пользователь,количество_участий,ФИО
0,ЕП,belov,1,Белов АП
1,ЕП,gagoc,4,Гагоц ВУ
2,ЕП,gegoc,6,gegoc
3,ЕП,gаgoc,1,gаgoc
4,ЕП,nov,2,nov
5,ЕП,vanov,4,Ванов КЕ
6,КУ,belov,6,Белов АП
7,КУ,dorov,2,dorov
8,КУ,sidorov,4,Сидоров АП
9,РП,belov,6,Белов АП


In [19]:
# Сортируем по услуге и количеству участий
rating_by_service = rating_by_service.sort_values(['код услуги', 'количество_участий'], ascending=[True, False])

print("Рейтинг по услугам (упрощенная версия):")
for service in rating_by_service['код услуги'].unique():
    print(f"\n--- Услуга: {service} ---")
    service_rating = rating_by_service[rating_by_service['код услуги'] == service]
    print(service_rating[['ФИО', 'пользователь', 'количество_участий']].to_string(index=False))

Рейтинг по услугам (упрощенная версия):

--- Услуга: ЕП ---
     ФИО пользователь  количество_участий
   gegoc        gegoc                   6
Гагоц ВУ        gagoc                   4
Ванов КЕ        vanov                   4
     nov          nov                   2
Белов АП        belov                   1
   gаgoc        gаgoc                   1

--- Услуга: КУ ---
       ФИО пользователь  количество_участий
  Белов АП        belov                   6
Сидоров АП      sidorov                   4
     dorov        dorov                   2

--- Услуга: РП ---
      ФИО пользователь  количество_участий
Петров ПП       petrov                  10
 Белов АП        belov                   6
    gegoc        gegoc                   4
 Гагоц ВУ        gagoc                   2
   ivonov       ivonov                   2
